<a href="https://colab.research.google.com/github/AronnePiperno/IML-Face-Search-Engine-La-Boccia/blob/master/Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torchvision.transforms as T
def get_data(batch_size, test_batch_size=256):
  
  # Prepare data transformations and then combine them sequentially
  transform = list()
  transform.append(T.ToTensor())                            # converts Numpy to Pytorch Tensor
  transform.append(T.Normalize(mean=[0.5], std=[0.5]))      # Normalizes the Tensors between [-1, 1]
  transform = T.Compose(transform)                          # Composes the above transformations into one.

  # Load data
  full_training_data = torchvision.datasets.MNIST('./data', train=True, transform=transform, download=True) 
  test_data = torchvision.datasets.MNIST('./data', train=False, transform=transform, download=True) 
  

  # Create train and validation splits
  num_samples = len(full_training_data)
  training_samples = int(num_samples*0.7+1)
  validation_samples = num_samples - training_samples

  training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])

  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=4)
  val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False, num_workers=4)
  test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=4)
  
  return train_loader, val_loader, test_loader


In [4]:
# Our network
import torch
class MLP(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(MLP, self).__init__()
    
    self.input_to_hidden = torch.nn.Linear(input_dim, hidden_dim)
    self.hidden_to_output = torch.nn.Linear(hidden_dim, output_dim)
    self.activation = torch.nn.Sigmoid()

  def forward(self, x):
    # Puts the output in (batch_size, input_dim) format
    x = x.view(x.shape[0],-1)

    # Forward input through the layers
    x = self.input_to_hidden(x)
    x = self.activation(x)
    x = self.hidden_to_output(x)
    return x

## Loss/cost function
For training the network, we obviously need a loss function. The task is classification with multiple classes, thus a proper loss could be a cross-entropy with softmax. We can again use `torch.nn` which contains several losses, among which `torch.nn.CrossEntropyLoss`. Notice that this loss already contains the softmax activation, thus we do not need to apply the softmax to the output of our network.

In [5]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

## Optimizer
Now we must devise a way to update the parameters of our network. This can be easily held out by having a look at [`torch.optim`](https://pytorch.org/docs/stable/optim.html) which contains a large variety of optimizers.

In [6]:
def get_optimizer(net, lr, wd, momentum):
  optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer

## Train and test functions
We are ready to merge everything by creating a training and test functions. Both of them must:

1.   Loop over the data (exploiting the dataloader, which is just an iterator)
2.   Forward the data through the network
3.  Comparing the output with the target labels for computing either the loss (train), the accuracy (test) or both.

Additionally, during training we must:


1.   Compute the gradient with the backward pass (`loss.backward()`)
2.   Using the optimizer to update the weights (`optimizer.step()`)
3.   Cleaning the gradient of the weights in order to not accumulating it (`optimizer.zero_grad()`)

With these steps in mind, we are ready to define everything.

We define:
*   **Iterations:** the number of gradients updates we are doing (i.e. number of times we call the `optimizer.step()`).
*   **Epoch:**number of times we iterate over the whole dataset.

Lets implement the training loop:

In [7]:
def train_one_epoch(net, data_loader, optimizer, cost_function, device='cuda'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  
  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU
    inputs = inputs.to(device)
    targets = targets.to(device)
      
    # Forward pass
    outputs = net(inputs)

    # Apply the loss
    loss = cost_function(outputs,targets)

    # Backward pass
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    # Reset the gradients
    optimizer.zero_grad()

    # Better print something, no?
    samples += inputs.shape[0]
    cumulative_loss += loss.item()
    
    _, predicted = outputs.max(dim=1) # max returns maximum_value, index_of_maximum_value
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100

While training, we need to test our model on the validaiton set to be sure we are improving over there as well

In [8]:
def validation_step(net, data_loader, cost_function, device='cuda'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)
        
      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      loss = cost_function(outputs, targets)

      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


## Wrapping everything up
Finally, we need a main function which initializes everything + the needed hyperparameters and loops over multiple epochs (printing the results).

In [9]:
import torchvision
import torch

def main(batch_size=128, input_dim=28*28, hidden_dim=100, output_dim=10, device='cuda:0', learning_rate=0.01, weight_decay=0.000001, momentum=0.9, epochs=10):

  # Gets DataLoaders
  train_loader, val_loader, test_loader = get_data(batch_size)
  
  # Istantiates the network and moves it to the chosen device (GPU)
  net = MLP(input_dim, hidden_dim, output_dim).to(device)
  
  # Instantiates the optimizer
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
  
  # Creates the cost function
  cost_function = get_cost_function()

  n_iterations = 0
  # For each epoch, train the network and then compute evaluation results
  for e in range(epochs):
    train_loss, train_accuracy = train_one_epoch(net, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = validation_step(net, val_loader, cost_function)

    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

In [12]:
main()

Epoch: 1
	 Training loss 0.00855, Training accuracy 74.75
	 Validation loss 0.00207, Validation accuracy 86.72
-----------------------------------------------------
Epoch: 2
	 Training loss 0.00337, Training accuracy 88.51
	 Validation loss 0.00154, Validation accuracy 89.39
-----------------------------------------------------
Epoch: 3
	 Training loss 0.00274, Training accuracy 90.08
	 Validation loss 0.00134, Validation accuracy 90.20
-----------------------------------------------------
Epoch: 4
	 Training loss 0.00245, Training accuracy 90.99
	 Validation loss 0.00122, Validation accuracy 91.12
-----------------------------------------------------
Epoch: 5
	 Training loss 0.00226, Training accuracy 91.68
	 Validation loss 0.00116, Validation accuracy 91.38
-----------------------------------------------------
Epoch: 6
	 Training loss 0.00212, Training accuracy 92.18
	 Validation loss 0.00109, Validation accuracy 92.08
-----------------------------------------------------
Epoch: 7
	

In [13]:
batch_size=128 
input_dim=28*28
hidden_dim=100
output_dim=10
device='cuda:0'
learning_rate=0.01
weight_decay=0.000001
momentum=0.9
epochs=10


# Gets DataLoaders
train_loader, val_loader, test_loader = get_data(batch_size)

# Istantiates the network and moves it to the chosen device (GPU)
net = MLP(input_dim, hidden_dim, output_dim).to(device)

# Instantiates the optimizer
optimizer = torch.optim.Adam(net.parameters())

# Creates the cost function
cost_function = get_cost_function()

n_iterations = 0
# For each epoch, train the network and then compute evaluation results
for e in range(epochs):
  train_loss, train_accuracy = train_one_epoch(net, train_loader, optimizer, cost_function)
  val_loss, val_accuracy = validation_step(net, val_loader, cost_function)

  print('Epoch: {:d}'.format(e+1))
  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('-----------------------------------------------------')

Epoch: 1
	 Training loss 0.00586, Training accuracy 83.46
	 Validation loss 0.00148, Validation accuracy 89.88
-----------------------------------------------------
Epoch: 2
	 Training loss 0.00250, Training accuracy 91.26
	 Validation loss 0.00112, Validation accuracy 91.92
-----------------------------------------------------
Epoch: 3
	 Training loss 0.00200, Training accuracy 92.78
	 Validation loss 0.00094, Validation accuracy 93.14
-----------------------------------------------------
Epoch: 4
	 Training loss 0.00171, Training accuracy 93.86
	 Validation loss 0.00084, Validation accuracy 93.77
-----------------------------------------------------
Epoch: 5
	 Training loss 0.00150, Training accuracy 94.58
	 Validation loss 0.00075, Validation accuracy 94.42
-----------------------------------------------------
Epoch: 6
	 Training loss 0.00133, Training accuracy 95.21
	 Validation loss 0.00067, Validation accuracy 94.96
-----------------------------------------------------
Epoch: 7
	

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
torch.save(net.state_dict(), "model.ckpt")

In [15]:
net.eval()

MLP(
  (input_to_hidden): Linear(in_features=784, out_features=100, bias=True)
  (hidden_to_output): Linear(in_features=100, out_features=10, bias=True)
  (activation): Sigmoid()
)

In [28]:
from PIL import Image
import os
from torchvision import transforms

transform = transforms.ToTensor()


results = {}
for file in os.listdir('/content/drive/MyDrive/mnist/images/'):
    img_name = file.split('.')[0]
    image = Image.open(os.path.join('/content/drive/MyDrive/mnist/images/', file))
    image = transform(image)
    image = image.to(device)
    output = net(image)
    #print(output)
    pred = output.argmax(dim=1, keepdim=True)[0]
    results[img_name] = pred.item()

print(results)

{'8376': 1, '5501': 6, '5504': 3, '5630': 1, '7957': 4, '7030': 7, '7882': 0, '5427': 2, '5526': 0, '6945': 8, '5983': 2, '7422': 1, '7888': 4, '7184': 2, '7139': 0, '7268': 7, '6496': 9, '6880': 5, '6613': 6, '5593': 0, '7077': 5, '6038': 6, '7910': 4, '6411': 9, '5307': 0, '7175': 0, '6853': 7, '6084': 2, '8197': 6, '5408': 3, '6914': 9, '5842': 9, '7738': 8, '8500': 4, '6710': 9, '5717': 2, '7031': 0, '8436': 9, '7039': 6, '5647': 8, '6681': 3, '5678': 8, '7414': 5, '7942': 9, '5723': 2, '5662': 5, '6803': 5, '8383': 9, '6610': 9, '6733': 1, '5447': 9, '5494': 2, '6158': 7, '5476': 4, '6761': 4, '7364': 2, '7388': 5, '5321': 0, '8375': 9, '5917': 1, '6250': 0, '7111': 7, '5518': 5, '5339': 5, '5229': 5, '5496': 7, '8492': 2, '6225': 2, '6075': 3, '7829': 3, '5485': 7, '8002': 9, '5859': 1, '6308': 1, '6468': 1, '5322': 4, '5275': 5, '5858': 7, '8486': 8, '6992': 7, '6654': 8, '5508': 3, '6918': 3, '6568': 4, '7761': 6, '7677': 7, '6414': 5, '6755': 8, '7659': 5, '7436': 3, '7737': 0

In [29]:
import requests
import json

# url localhost
url = "http://coruscant.disi.unitn.it:3001/results/"

def submit(results, url="http://coruscant.disi.unitn.it:3001/results/"):
    res = json.dumps(results)
    # print(res)
    response = requests.post(url, res)
    try:
        result = json.loads(response.text)
        print(f"accuracy is {result['accuracy']}")
        print(f"precision is {result['precision']}")
        print(f"recall is {result['recall']}")
    except json.JSONDecodeError:
        print(f"ERROR: {response.text}")

In [34]:
query = dict()
query['groupname'] = "La Boccia"

query["images"] = results

print(query)

{'groupname': 'La Boccia', 'images': {'8376': 1, '5501': 6, '5504': 3, '5630': 1, '7957': 4, '7030': 7, '7882': 0, '5427': 2, '5526': 0, '6945': 8, '5983': 2, '7422': 1, '7888': 4, '7184': 2, '7139': 0, '7268': 7, '6496': 9, '6880': 5, '6613': 6, '5593': 0, '7077': 5, '6038': 6, '7910': 4, '6411': 9, '5307': 0, '7175': 0, '6853': 7, '6084': 2, '8197': 6, '5408': 3, '6914': 9, '5842': 9, '7738': 8, '8500': 4, '6710': 9, '5717': 2, '7031': 0, '8436': 9, '7039': 6, '5647': 8, '6681': 3, '5678': 8, '7414': 5, '7942': 9, '5723': 2, '5662': 5, '6803': 5, '8383': 9, '6610': 9, '6733': 1, '5447': 9, '5494': 2, '6158': 7, '5476': 4, '6761': 4, '7364': 2, '7388': 5, '5321': 0, '8375': 9, '5917': 1, '6250': 0, '7111': 7, '5518': 5, '5339': 5, '5229': 5, '5496': 7, '8492': 2, '6225': 2, '6075': 3, '7829': 3, '5485': 7, '8002': 9, '5859': 1, '6308': 1, '6468': 1, '5322': 4, '5275': 5, '5858': 7, '8486': 8, '6992': 7, '6654': 8, '5508': 3, '6918': 3, '6568': 4, '7761': 6, '7677': 7, '6414': 5, '6755

In [35]:
submit(query)

ConnectionError: ignored